In [ ]:
path = []


In [ ]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [ ]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

In [ ]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

In [ ]:
import math
print(slp)

In [3]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np

In [4]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost
minibatch = []

In [59]:
data = np.random.rand(100,15) # 5 entities, each contains 10 features
label = np.random.randint(6, size=100)
dtest = xgb.DMatrix([data[0]])
print(label.shape)
DQNAgent().xgbmodel_bld(data,label)
DQNAgent().bst.predict(dtest)
# np.argmax([DQNAgent().bst.predict(dtest),10])

(100,)
[0]	train-merror:0.41	eval-merror:0.41
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.33	eval-merror:0.33
[2]	train-merror:0.25	eval-merror:0.25
[3]	train-merror:0.25	eval-merror:0.25
[4]	train-merror:0.26	eval-merror:0.26
[5]	train-merror:0.25	eval-merror:0.25
[6]	train-merror:0.19	eval-merror:0.19
[7]	train-merror:0.19	eval-merror:0.19
[8]	train-merror:0.18	eval-merror:0.18
[9]	train-merror:0.16	eval-merror:0.16
[10]	train-merror:0.17	eval-merror:0.17
[11]	train-merror:0.15	eval-merror:0.15
[12]	train-merror:0.15	eval-merror:0.15
[13]	train-merror:0.13	eval-merror:0.13
[14]	train-merror:0.13	eval-merror:0.13
[15]	train-merror:0.13	eval-merror:0.13
[16]	train-merror:0.13	eval-merror:0.13
[17]	train-merror:0.12	eval-merror:0.12
[18]	train-merror:0.13	eval-merror:0.13
[19]	train-merror:0.12	eval-merror:0.12
[20]	train-merror:0.13	eval-merror:0.13
[21]	train-merro

array([ 0.5], dtype=float32)

In [75]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = xgb.Booster({'nthread':4})
        self.name = 'steerDeepQL'
        self.modelname = 'xgbmodel'
        self.load_bst()
    
    def xgbmodel_bld(self,data,label):
        param = {'base_score' : 1, 'max_depth': 7, 'eta': 0.1, #'updater':'refresh',
            #'process_type': 'update',
            'refresh_leaf': True,
            'reshape': True,
            'reg_alpha': 3, 
            'silent': 1, 
            'objective': 'multi:softprob',     
            'num_class': 6}  
        param['nthread'] = 4
        param['eval_metric'] = 'merror'
        num_round = 200  
        dtrain = xgb.DMatrix(data, label=label)
        evallist = [(dtrain, 'train'), (dtrain, 'eval')]
        self.bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10, xgb_model = self.name)
        self.save_bst()

        
    def load_bst(self):
        self.bst.load_model(self.name)
    
    def save_bst(self):
        self.bst.save_model(self.name)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(xgb.DMatrix(state))
        print('act_values',act_values)
        return np.argmax(act_values[0])  
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        minibatch = self.memory
        print(len(minibatch))
        bst_target = xgb.Booster({'nthread':4})
        bst_target.load_model(self.name)
        
        for state, action, reward, next_state, done in minibatch:
#             print('state',state)
            target = bst_target.predict(xgb.DMatrix(state))##target is Q matrix
#             print('target',target)
#             print('atc',action)
#             print('reward',reward)
            if done:
                target[0][action] = reward
            else:
                t = self.bst.predict(xgb.DMatrix(next_state))[0]
                target[0][action] = reward + self.gamma * np.amax(t)
#             print(target)
#             print(np.argmax(target[0]))
            self.xgbmodel_bld(state, [np.argmax(target[0])])
            self.xgbmodel_bld(state, [np.argmax(target[0])])

        self.memory.clear()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1]))
        done = False
        if rew > 130:
            done = True
        return rew,done
    
    def get_nextstate(self):
        ########
        stt = np.random.randint(15, size=15)
        return stt
    
def steer(GameOn):
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((15,), dtype=float)
    state = agent.get_nextstate()
#     agent.load_bst()
    
    while GameOn:
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
        print("action:",action)
#         execute_action(action)
        reward, done = agent.decl_rew(state)
        reward = reward if not done else -10
        next_state = agent.get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if len(agent.memory) >= 500 :
            print("replaying at {} xgboost".format(len(agent.memory)))
            agent.replay()
    
        

In [76]:
steer(True)

action: 2
action: 2
action: 1
action: 5
action: 3
action: 0
action: 2
action: 4
action: 2
action: 1
action: 4
action: 0
action: 4
action: 3
action: 4
action: 3
action: 4
action: 1
action: 3
action: 0
action: 1
action: 4
action: 2
action: 1
action: 1
action: 4
action: 3
action: 1
action: 4
action: 0
action: 4
action: 2
action: 1
action: 5
action: 3
action: 3
action: 5
action: 0
action: 4
action: 4
action: 2
action: 4
action: 4
action: 1
action: 1
action: 1
action: 5
action: 4
action: 4
action: 2
action: 2
action: 3
action: 5
action: 4
action: 2
action: 5
action: 3
action: 2
action: 2
action: 0
action: 3
action: 2
action: 2
action: 3
action: 2
action: 0
action: 2
action: 1
action: 1
action: 3
action: 3
action: 4
action: 0
action: 2
action: 4
action: 1
action: 3
action: 4
action: 5
action: 1
action: 0
action: 4
action: 4
action: 2
action: 0
action: 3
action: 2
action: 5
action: 3
action: 4
action: 5
action: 0
action: 3
action: 3
action: 1
action: 1
action: 4
action: 5
action: 4
action: 0


/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:106: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0

[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed

[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval

...

[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:1	eval-merror:1
Multiple eval metrics have been passed

[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:1	eval-merror:1
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:1	eval-merror:1
[2]	train-merror:1	eval-merror:1
[3]	train-merror:1	eval-merror:1
[4]	train-merror:1	eval-merror:1
[5]	train-merror:1	eval-merror:1
[6]	train-merror:1	eval-merror:1
[7]	train-merror:1	eval-merror:1
[8]	train-merror:1	eval-merror:1
[9]	train-merror:1	eval-merror:1
[10]	train-merror:1	eval-merror:1
Stopping. Best iteration:
[0]	train-mer

[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval-merror:0
[2]	train-merror:0	eval-merror:0
[3]	train-merror:0	eval-merror:0
[4]	train-merror:0	eval-merror:0
[5]	train-merror:0	eval-merror:0
[6]	train-merror:0	eval-merror:0
[7]	train-merror:0	eval-merror:0
[8]	train-merror:0	eval-merror:0
[9]	train-merror:0	eval-merror:0
[10]	train-merror:0	eval-merror:0
Stopping. Best iteration:
[0]	train-merror:0	eval-merror:0

[0]	train-merror:0	eval-merror:0
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0	eval

KeyboardInterrupt: 

In [ ]:
minibatch

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
import random
import gym
import numpy as np
from collections import deque
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam
from keras import backend as K

EPISODES = 500


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        
    
    def _build_model(self):
        param = { 'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 6}  
        num_round = 2000  
        classifier = xgb.sklearn.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
                        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
                       objective='multi:softmax', reg_alpha=0, reg_lambda=1,
                       scale_pos_weight=1, seed=42, silent=True, subsample=1)
        return classifier
    
    def update_target_model(self):
        self.target_model.load_model(self.model.save_model())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) 
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
#         eval_set = [(X_test, y_test)]
        self.model.fit(state, target,eval_metric="error", eval_set=eval_set, verbose=True)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_model(name)
    
    def save(self, name):
        self.model.save_model(name)
    
    def decl_rew(self,state):
        rew = np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1]))
        d = False
        if rew > 130:
            d = True
        return [nxtstate, rew, True]
        


def steer():
    state_size = 15
    action_size = 6
    agent = DQNAgent(state_size, action_size)
    
#     agent.load_model("./save/steerwts.h5")
    
    batch_size = 50
    state = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] 
    while True:
        done = False
        while True:
            state = np.reshape(state, [1, state_size])
            action = agent.act(state)
            print("action:",action)
            execute_action(action)
            next_state, reward, done = decl_rew(state)##input and reward
            if(next_state == []):
                continue;
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("updating weights under 10 deg")
                agent.update_target_model()
            if len(agent.memory) > batch_size:
                print("replaying at 50 and saving weights")
                agent.replay(batch_size)

In [ ]:
import numpy as np
from PIL import ImageGrab
import cv2
import time

def screen_record(): 
    last_time = time.time()
    while True:
        # 800x600 windowed mode
        printscreen_pil =  ImageGrab.grab(bbox=(0,40,800,640))
        printscreen_numpy =   np.array(printscreen_pil.getdata(),dtype='uint8')\
        .reshape((printscreen_pil.size[1],printscreen_pil.size[0],3))
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()

    ##    cv2.imshow('window',cv2.cvtColor(printscreen_numpy, cv2.COLOR_BGR2RGB))
    ##    if cv2.waitKey(25) & 0xFF == ord('q'):
    ##        cv2.destroyAllWindows()
    ##        break

In [54]:
import numpy as np
import xgboost as xgb
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softmax','num_class': 3}  
num_round = 200  
bst = xgb.train(param, dtrain, num_round,evals=[(dtrain, "training")])#,xgb_model='mm')
# preds = bst.predict(dtest)

# best_preds = np.asarray([np.argmax(line) for line in preds])
# print("Numpy array precision:", precision_score(y_test, best_preds, average='macro') )


# joblib.dump(bst, 'bst_modellll', compress=True)



[0]	training-merror:0.033333
[1]	training-merror:0.033333
[2]	training-merror:0.033333
[3]	training-merror:0.025
[4]	training-merror:0.025
[5]	training-merror:0.025
[6]	training-merror:0.025
[7]	training-merror:0.016667
[8]	training-merror:0.016667
[9]	training-merror:0.016667
[10]	training-merror:0.016667
[11]	training-merror:0.016667
[12]	training-merror:0.016667
[13]	training-merror:0.016667
[14]	training-merror:0.016667
[15]	training-merror:0.016667
[16]	training-merror:0.016667
[17]	training-merror:0.008333
[18]	training-merror:0
[19]	training-merror:0
[20]	training-merror:0
[21]	training-merror:0
[22]	training-merror:0
[23]	training-merror:0
[24]	training-merror:0
[25]	training-merror:0
[26]	training-merror:0
[27]	training-merror:0
[28]	training-merror:0
[29]	training-merror:0
[30]	training-merror:0
[31]	training-merror:0
[32]	training-merror:0
[33]	training-merror:0
[34]	training-merror:0
[35]	training-merror:0
[36]	training-merror:0
[37]	training-merror:0
[38]	training-merror:0

In [49]:
bst.save_model('mm')

In [36]:
bst1 = xgb.Booster({'nthread':4}) #init model
bst1.load_model("m")

In [57]:
bst.predict(dtest)

array([ 1.,  0.,  2.,  1.,  1.,  0.,  1.,  2.,  1.,  1.,  2.,  0.,  0.,
        0.,  0.,  1.,  2.,  1.,  1.,  2.,  0.,  2.,  0.,  2.,  2.,  2.,
        2.,  2.,  0.,  0.], dtype=float32)

In [30]:
xgb1 = joblib.load('m')
xgb.train(param, dtrain, num_round)#,xgb_model='bst_model')
xgb1.predict(dtest)
# joblib.dump(xgb1, 'bst_modell.pkl', compress=True)

FileNotFoundError: [Errno 2] No such file or directory: 'm'

In [ ]:
bst1

In [18]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

EPISODES = 5000


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
    
    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)
    
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
#             print(target)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-ddqn.h5")
    done = False
    batch_size = 32
    
    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
# if e % 10 == 0:
#     agent.save("./save/cartpole-ddqn.h5")



[2018-05-27 11:47:29,492] Making new env: CartPole-v1


episode: 0/5000, score: 13, e: 1.0
episode: 1/5000, score: 39, e: 1.0


AttributeError: 'Sequential' object has no attribute 'prob_predict'